# Tabulate fit quality with different models

## Caution
* TODO: need to check that every result converged
* TODO: Plot a few samples
* TODO: need to figure out appropriate start and finish points
* (I may prototype an automatic interval finder)


In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import fitting as fit
import data_utils as dat
import pandas as pd
import vis
import plotly.graph_objects as go

In [ ]:
msizes = {
    '14m': (256, 80000),
    '31m': (256, 80000),
    '70m': (256, 80000),
    '160m': (256, 80000),
    '410m': (256, 80000),
    '1b': (256, 80000)
}
functions = [
    fit.ShiftedPowerLaw2,
    fit.DoubleShiftedPowerLaw,
    fit.DoubleExponential,
]
metrics = {
    "R2_log": fit.logspace_r2,
    "R2_lin": fit.r2_score,
    "RMSE": fit.rmse,
}
compute_diagnostics = False

reports = []


for msize, (start_step, end_step) in msizes.items():
    df_llc, df_loss = dat.load_dfs(msize, data_path="data")
    report = []
    
    for task in df_llc:
        for function in functions:
            x, y, s = dat.trim_trace(df_llc, df_loss, task, start_step, end_step)
            result = fit.min_fit(x, y, function)
        
            # Evaluate
            row = {
                "Model": "Pythia" + msize,
                "Dataset": task,
                "Function": function.name,
            }
            # Actually don't store the parameters because they don't match across methods
            # row.update(result.params_dict)
            y_pred = result.f(x)
           
            measures = {k: v(y, y_pred) for k, v in metrics.items()}
            row.update(measures)
            
            if compute_diagnostics:
                row.update(result.pcov_diagnostics())
    
            report.append(row)
            
    report = pd.DataFrame(report, index=range(len(report)))
    display(report)
    reports.append(report)

# Conclusion - all these models do really well - if we spot a low score perhaps plot it and debug - its likely initial parameters or how we're trimming the series
